<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/dev/examples/AI_Generated_Voiceovers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔊 Adding AI Generated voiceovers to silent footage

## Overview
Voiceovers are the secret sauce that turns silent footage into captivating stories. They add depth, emotion, and excitement, elevating the viewing experience to new heights. With [VideoDB](https://videodb.io)'s cutting-edge technology, creating dynamic voiceovers for your videos is easier and more thrilling than ever before.

Creating AI content and merging outputs from different tools can be tough and time-consuming. Doing it manually is even harder! What if you could do it all- in just a few lines of code? 

[VideoDB](https://videodb.io)'s cool tech helps you generate intelligent AI content and combine outputs from various tools quickly and automatically. No more long and tedious processes. Just simple, efficient, and awesome results


Let’s try adding an automatically generated voiceover to this silent footage in the style and voice of Sir David Attenborough. We shall do this using the powerful tech provided by [Open AI](https://openai.com), [ElevenLabs](https://elevenlabs.io) and [VideoDB](https://videodb.io)



## Setup

### Installing the required packages

In [ ]:
%pip install openai
%pip install videodb

### API keys
Before proceeding, ensure access to [VideoDB](https://videodb.io), [OpenAI](https://openai.com), and [ElevenLabs](https://elevenlabs.io) API key. If not, sign up for API access on the respective platforms.

> You can get VideoDB's API key from 👉🏼 [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required!** )

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["ELEVEN_LABS_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

### ElevenLab's Voice ID 

You will also need ElevenLab's VoiceID of a Voice that you want to use.

For this demo, we will be using David Attenborough's Voice.   
Please add [this](https://elevenlabs.io/app/voice-lab/share/eea2654def1e6c5bda5b4ce8f99f8f2c857b71a15cd6188c27d337206ea98177/s6sJbrmNIsT6M7vdjjES) Voice to Your VoiceLab and copy VoiceID from there.



In [ ]:
voiceover_artist_id = "VOICEOVER_ARTIST_ID"


### 🌐 Step 1: Connect to VideoDB
Connect to VideoDB using your API key to establish a session for uploading and manipulating video files.

In [5]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()

### 🎥 Step 2: Upload Video

In [8]:
video_url = "https://youtu.be/RcRjY5kzia8"
video = conn.upload(url=video_url)

### 🔍 Step 3: Analyze Scenes and Generate Scene Descriptions

Start by analyzing the scenes within your Video using VideoDB's scene indexing capabilities. This will provide context for generating the script prompt.

In [10]:
video.index_scenes()

In [7]:
scenes = video.get_scenes()

Let's view the description of first scene from the video

In [10]:
print(f"{scenes[0]['start']} - {scenes[0]['end']}")
print(scenes[0]["response"])

0 - 9.033333333333333
The image presents a close-up, textured pattern reminiscent of organic forms. Dominated by a cool color palette of blue and turquoise hues, it gives the impression of looking at a magnified cluster of cells, crystalline structures, or perhaps a zoomed-in portion of aquatic flora. The shapes appear irregular yet symmetrical, like petals or leaves clustered tightly together. Light and shadow play across the surfaces, creating a dynamic interplay that suggests depth and complexity. The overall effect is one of natural beauty, with a soothing, almost hypnotic visual rhythm. The image is abstract enough to allow for multiple interpretations, depending on the viewer's perspective.


### 🎤 Step 4: Generate Voiceover Script with ChatGPT
Combine scene descriptions with the script prompt, instructing ChatGPT to create a voiceover script in David Attenborough's style.

This script prompt can be refined and tweaked to generate the most suitable output. Check out these examples to explore more usecases.

In [30]:
import openai

client = openai.OpenAI()


def generate_voiceover_script(scenes, script_prompt):
    full_prompt = script_prompt + "\n\n"
    for scene in scenes:
        full_prompt += f"- {scene}\n"

    openai_res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": full_prompt}],
    )

    return openai_res


script_prompt = "Here's the data from a scene index for a video about the underwater world. Study this and then generate a synced script based on the description below. Make sure the script is in the language, voice and style of Sir David Attenborough"

openai_res = generate_voiceover_script(scenes=scenes, script_prompt=script_prompt)
voiceover_script = openai_res.choices[0].message.content

# If you have ElevenLab's paid plan remove this
voiceover_script = voiceover_script[:2500]

### 🎤 Step 5: Generate Voiceover Audio with ElevenLabs
Utilize the generated script to synthesize AI-generated voiceover narration in David Attenborough's voice using ElevenLabs API.

In [31]:
import requests


# Call ElevenLabs API to generate voiceover
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voiceover_artist_id}"
headers = {
    "xi-api-key": os.environ.get("ELEVEN_LABS_API_KEY"),
    "Content-Type": "application/json"
}
payload = {
    "model_id": "eleven_monolingual_v1",
    "text": voiceover_script,
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5
    }
}
elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)

# Save the audio file
audio_file = "audio.mp3"
CHUNK_SIZE = 1024
with open(audio_file, 'wb') as f:
    for chunk in elevenlabs_res.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

### 🎬 Step 6: Add Voiceover to Video with VideoDB

In order to use the voiceover generated above, let's upload the audio file (voiceover) to VideoDB first

In [32]:
audio = conn.upload(file_path=audio_file)

Finally, add the AI-generated voiceover to the silent footage of the underwater world using VideoDB's [timeline feature](https://docs.videodb.io/version-0-0-3-timeline-and-assets-44).

In [ ]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset

# Create a timeline object
timeline = Timeline(conn)

# Add the video asset to the timeline for playback
video_asset = VideoAsset(asset_id=video.id)
timeline.add_inline(asset=video_asset)

# Add the audio asset to the timeline for playback
audio_asset = AudioAsset(asset_id=audio.id)
timeline.add_overlay(start=0, asset=audio_asset)

### 🪄 Step 7: Review and Share
Preview the video with the integrated voiceover to ensure it functions correctly.   
Once satisfied, generate a stream of the video and share the link for others to view and enjoy this wholesome creation!


In [33]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/5e39f02c-3886-4cbc-a193-b6f672acbaab.m3u8'

### 🎉 Conclusion:
Congratulations! You have successfully automated the process of creating custom and personalised voiceovers based on a simple prompt and raw video footage using VideoDB, ChatGPT, and ElevenLabs. By leveraging advanced AI technologies, you can enhance the storytelling and immersive experience of your video content. Experiment with different prompts and scene analysis techniques to further improve the quality and accuracy of the voiceovers. Enjoy creating captivating narratives with AI-powered voiceovers using VideoDB! We're excited to see your creations so you're welcome to share them on our Discord channel [here](https://discord.com/invite/py9P639jGz) 